In [1]:
import os
import numpy as np
import pandas as pd

from quantifier_recommender import QuantifierRecommender

quantifier_recommender = QuantifierRecommender()

# quantifier_recommender.construct_meta_table(datasets_path="./datasets/",
#                                             train_data_path="./data/train_data/",
#                                             test_data_path="./data/test_data/",
#                                             supervised=True)

# quantifier_recommender.evaluation_table.to_csv("./recommender_evaluation_table.csv", index=False)
# quantifier_recommender.meta_features_table.to_csv("./recommender_meta_features_table.csv", index=False)

df = pd.read_csv("./recommender_evaluation_table2.csv")

In [39]:
df = pd.read_csv('./test.csv')

iterables = [df['quantifier'].unique().tolist(), df['dataset'].unique().tolist()]
index = pd.MultiIndex.from_product(iterables, names=["quantifier", "dataset"])

abs_error_list = df["abs_error"].tolist()
run_time_list = df["run_time"].tolist()
new_df = pd.DataFrame({"abs_error": abs_error_list, "run_time": run_time_list}, index=index)
# df.pop('quantifier')
# df.pop('dataset')

# df

new_df.loc['ACC', 'AedesSex']['abs_error']

# X_train = meta_features.values
# y_train = new_df.loc['DyS']['abs_error'].values

# # CC
# fit(X_train, y_train)

# rec_dict = {"CC": rec, "ACC": rec}

# dataset_novo -> extrai meta features -> linha de metafeatures




# y_train
# # # index.get_loc()

np.float64(0.0144999999999999)

In [73]:
import os
import numpy as nps
import pandas as pd
import pdb

from quantifier_recommender import QuantifierRecommender

qtf_rec = QuantifierRecommender.load_model("s_qtf_recommender.pkl")
evaluation_table = qtf_rec.evaluation_table

# recommenders = qtf_rec.rec

y_test = None
y_train = None

# recommender_evaluation_table = pd.DataFrame(columns=)
for quantifier, recommender in qtf_rec.recommender_dict.items():
    for dataset in qtf_rec.evaluation_table.index.levels[1]:
        X_test = qtf_rec.meta_features_table.loc[dataset].values
        y_test = qtf_rec.evaluation_table.loc[quantifier, dataset]['abs_error']

        X_train = qtf_rec.meta_features_table.drop(index=dataset).values
        y_train = qtf_rec.evaluation_table.loc[quantifier].drop(index=dataset)['abs_error'].values

        
        
        break
    break
#     for quantifier in self.evaluation_table.index.levels[0].tolist():
#         y_train = self.evaluation_table.loc[quantifier]['abs_error'].values
#         self.recommender_dict[quantifier] = RandomForestRegressor()
#         self.recommender_dict[quantifier].fit(X_train, y_train)
#         print(index)
        # test_data = evaluation_table.loc[index]
        # train_data = evaluation_table.drop(index=index)
        # break



# train_data
# y_train
X_train

array([[0.00000000e+00, 0.00000000e+00, 9.99553608e-01, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 6.92588185e-02, 3.74385367e-02,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        6.88227916e-01, 3.53067975e-01, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        9.38711916e-01, 3.42879745e-02, 1.00000000e+00, 0.00000000e+00,
        7.28612362e-02, 3.59383549e-01, 6.54244851e-01, 3.91324803e-01,
        3.84688156e-01, 4.48344695e-05, 7.29761152e-06, 0.00000000e+00,
        2.00411445e-01, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [53]:
qtf_rec.evaluation_table

abs_error  run_time
quantifier dataset                         
ACC        AedesQuinx      0.0465  0.001329
           AedesSex        0.0145  0.001281
           anuranCalls     0.0200  0.001361
CC         AedesQuinx      0.0920  0.000008
           AedesSex        0.0190  0.000010
           anuranCalls     0.0400  0.000010
DyS        AedesQuinx      0.0425  0.049736
           AedesSex        0.0075  0.048827
           anuranCalls     0.0140  0.048127
HDy        AedesQuinx      0.0605  0.219288
           AedesSex        0.0085  0.217841
           anuranCalls     0.0260  0.208500
MAX        AedesQuinx      0.0480  0.000203
           AedesSex        0.0130  0.000209
           anuranCalls     0.0165  0.000203
MS         AedesQuinx      0.0460  0.030952
           AedesSex        0.0130  0.031222
           anuranCalls     0.0185  0.031122
PACC       AedesQuinx      0.0795  0.004147
           AedesSex        0.0220  0.004062
           anuranCalls     0.0220  0.004088
PCC        AedesQuinx      0.1440  0.002745
           AedesSex        0.0295  0.002721
           anuranCalls     0.0545  0.002634
SMM        AedesQuinx      0.0460  0.000151
           AedesSex        0.0100  0.000149
           anuranCalls     0.0195  0.000115
SORD       AedesQuinx      0.0420  1.379702
           AedesSex        0.0115  1.385671
           anuranCalls     0.0175  0.361445
X          AedesQuinx      0.0475  0.000423
           AedesSex        0.0145  0.000419
           anuranCalls     0.0210  0.000398

In [26]:
y_train = qtf_rec.evaluation_table.loc[quantifier]['abs_error'].values
y_train

array([0.0465, 0.0145, 0.02  ])

In [23]:
qtf_rec.recommender_dict.keys()

dict_keys(['ACC', 'CC', 'DyS', 'HDy', 'MAX', 'MS', 'PACC', 'PCC', 'SMM', 'SORD', 'X'])

In [16]:
evaluation_table['ACC']["AedesQuinx"]

KeyError: 'ACC'